# Installation:

### Install using pip:

pip install ipython-cluster-helper

#### If you receive an error stating "No module named 'pip.req'", you may need to change your pip version to 9.0.1 by running:

pip install --upgrade pip==9.0.1


# Import

In [ ]:
from cluster_helper.cluster import cluster_view

# Basic Example

In [ ]:
# Define the function you wish to run multiple copies of
def square(N):
    y = N**2

# Define the list of inputs you wish to process in parallel
inputs = [1, 2, 3, 4]

In [ ]:
"""
Submit parallel jobs using cluster helper. Cluster helper will automatically run the function you provide on each of 
the inputs you provide. Note that your function can only take one argument, or it will not work with cluster helper. 
You should design your function in a way that it will not require more than one input.

Parameters include:
scheduler - Rhino uses the Sun Grid Engine, so set this to 'sge'
queue - 'RAM.q' is the main queue on Rhino. We also have 'matlab.q' to place jobs on nodes with MATLAB licenses.
num_jobs - The number of parallel jobs you want to run. 
cores_per_job - The number of cores assigned to each job. Usually leave this set to 1, but it can be used to limit 
the number of jobs assigned to the same node in order to avoid memory errors.
"""
with cluster_view(scheduler='sge', queue='RAM.q', num_jobs=4, cores_per_job=1) as view:
    view.map(square, inputs)

# EEG Example

In [ ]:
# This function will read the header of an EEG file and write the time of the recording to a text file
def read_eeg_time(eegfile):
    # When using cluster helper, your function will only have access to packages you import in the function itself,
    # so we will import os and mne here instead of at the top of the file.
    import os
    import mne
    
    if os.path.exists(eegfile):
        raw = mne.io.read_raw_edf(eegfile, preload=False)
        t = str(raw.info['meas_date'])

        outfile = '/scratch/jpazdera/' + os.path.splitext(os.path.basename(eegfile))[0] + '_time.txt'
        with open(outfile, 'w') as f:
            f.write(t)

In [ ]:
# Set up a list of EEG file paths we want to work with
eegfiles = []
subj = 'LTP331'
for sess in range(3):
    filepath = '/protocols/ltp/subjects/%s/experiments/ltpFR2/sessions/%s/ephys/current_processed/%s_session_%s.bdf' % (subj, sess, subj, sess)
    eegfiles.append(filepath)

In [ ]:
with cluster_view(scheduler='sge', queue='RAM.q', num_jobs=3, cores_per_job=1) as view:
    view.map(read_eeg_time, eegfiles)